In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 데이터 로드
data = 'GRU/portfolio_weights_GRU_no_prob_sharpe_ratio.csv'
df = pd.read_csv(data, index_col=0)
df.index = pd.to_datetime(df.index)
df

# 2018-03-29 이후 데이터 선택
df = df.loc['2018-03-29':]

# 20 거래일 간격으로 인덱스 생성
indices = np.arange(0, len(df), 20)
df = df.iloc[indices]
df

# 소수점 4자리 이하는 버림
df = df.round(4)
df

# 0이 아닌 칼럼만 선택
df = df.loc[:, (df != 0).any()]
df 

,BK
2018-03-29,0.9995
2018-04-27,0.9995
2018-05-25,0.9995
2018-06-25,0.9995
2018-07-24,0.9995
...,...
2024-02-15,0.9995
2024-03-15,0.9995
2024-04-15,0.9995
2024-05-13,0.9995


In [ ]:

return_path = '../../Data/processed/return_df.csv'
return_df = pd.read_csv(return_path, index_col=0)
return_df.index = pd.to_datetime(return_df.index)

# df의 인덱스를 기준으로 return_df 재정렬
return_df = return_df.reindex(df.index)

# 월별 데이터로 리샘플링
df_monthly = df.resample('M').last()
return_df_monthly = return_df.resample('M').last()

# 월별 누적 수익률 계산
portfolio_return = (df_monthly * return_df_monthly).sum(axis=1)
cumulative_return = (1 + portfolio_return).cumprod()

# 첫 번째 그래프: 월별 누적 수익률
plt.figure(figsize=(12, 6))
cumulative_return.plot()
plt.title('Monthly Cumulative Return')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()

# 상위 20% 수익률 종목 분석
def calculate_top_holdings(weights, returns, percentile=0.8):
    """
    각 월별로 상위 20% 수익률을 기록한 종목들의 보유 비중을 계산
    """
    top_stocks = returns.rank(axis=1) >= len(returns.columns) * percentile
    top_holdings = (weights * top_stocks).sum(axis=1)
    return top_holdings

# 월별 상위 20% 종목 보유 비중 계산
top_holdings = calculate_top_holdings(df_monthly, return_df_monthly)

# 두 번째 그래프: 월별 상위 20% 수익률 종목 보유 비중
plt.figure(figsize=(12, 6))
top_holdings.plot()
plt.title('Monthly Top 20% Return Stocks Holding')
plt.xlabel('Date')
plt.ylabel('Holding')
plt.grid(True)
plt.show()